In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load depth image (grayscale)
depth_map = cv2.imread("depth_map.jpg", cv2.IMREAD_GRAYSCALE)

In [ ]:
# Resize for faster processing (optional)
scale = 1
depth_map = cv2.resize(depth_map, (0, 0), fx=scale, fy=scale)

h, w = depth_map.shape

In [ ]:
# Grid step
step = 5  # adjust spacing
x_points = np.arange(0, w, step)
y_points = np.arange(0, h, step)

In [ ]:
# Generate grid
X, Y = np.meshgrid(x_points, y_points)
Z = []

In [ ]:
# Normalize depth: 0 (black) => 1.0 (far), 255 (white) => 0.0 (close)
for y in y_points:
    row = []
    for x in x_points:
        intensity = depth_map[y, x]
        z = 1.0 - (intensity / 255.0)  # inverse: black => 1.0, white => 0.0
        row.append(z * 500)  # scale Z for visibility
    Z.append(row)


Z = np.array(Z)

In [ ]:
# Plot the grid in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z, c=Z, cmap='viridis', s=5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_box_aspect([np.ptp(X), np.ptp(Y), np.ptp(Z)])

plt.title("3D Grid from Depth Map")
plt.show()

In [ ]:
fig = plt.figure(figsize=(40, 20))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z, c=Z, cmap='viridis', s=5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_box_aspect([np.ptp(X), np.ptp(Y), np.ptp(Z)])
plt.title("3D Grid from Depth Map")

# Set view angle
ax.view_init(elev=360-90, azim=90)

plt.show()